In [1]:
import pandas as pd
import xml.etree.ElementTree as ET
import geopandas
import numpy as np
from shapely.geometry import Point
import copy

In [2]:
def read_vrp_solution(file_name):
    """Reads solution from output XML file of jsprit
    :return:  JspritSolution
    """
    tree = ET.parse(file_name)
    root = tree.getroot()
    namespace = {'xmlns': 'http://www.w3schools.com'}
    solutions_element = root.find('xmlns:solutions', namespace)
    solution_element = solutions_element.find('xmlns:solution', namespace)
    routes = []
    routes_element = solution_element.find('xmlns:routes', namespace)
    # Theoretically there could be situation when no traveler can be routed
    if routes_element is None:
        return None
    for route_element in routes_element.findall('xmlns:route', namespace):
        acts = []
        for act_element in route_element.findall('xmlns:act', namespace):
            person_id_element = act_element.find('xmlns:shipmentId', namespace)
            if person_id_element is None:
                person_id_element = act_element.find('xmlns:serviceId', namespace)
            act = JspritAct(type_=JspritAct.get_type_from_string(act_element.attrib.get('type')),
                            person_id=int(person_id_element.text),
                            end_time=float(act_element.find('xmlns:endTime', namespace).text),
                            arrival_time=float(act_element.find('xmlns:arrTime', namespace).text)
                            )
            acts.append(act)
        route = JspritRoute(vehicle_id=int(route_element.find('xmlns:vehicleId', namespace).text),
                            start_time=float(route_element.find('xmlns:start', namespace).text),
                            end_time=float(route_element.find('xmlns:end', namespace).text),
                            acts=acts
                            )
        routes.append(route)
    unassigned_jobs_elements = solution_element.findall('xmlns:unassignedJobs', namespace)
    unassigned_job_ids = []
    # there could be unroutable or undeliverable requests
    if unassigned_jobs_elements is not None:
        if len(unassigned_jobs_elements) != 0:
            for unassigned_jobs_element in unassigned_jobs_elements[0].findall('xmlns:job', namespace):
                unassigned_job_ids.append(int(unassigned_jobs_element.attrib.get('id')))

    solution = JspritSolution(cost=float(solution_element.find('xmlns:cost', namespace).text),
                              routes=routes,
                              unassigned=unassigned_job_ids)
    return solution

In [3]:
class ActType(object):
    PICK_UP = 0
    DROP_OFF = 1
    DELIVERY = 2
    DRIVE = 3
    WAIT = 4
    RETURN = 5
    IDLE = 6

    def __init__(self, type_=None):
        self.type = type_

    @staticmethod
    def get_type_from_string(act_string):
        return {'pickupShipment': ActType.PICK_UP,
                'deliverShipment': ActType.DROP_OFF,
                'delivery': ActType.DELIVERY
                }[act_string]

    @staticmethod
    def get_string_from_type(act_type):
        return {ActType.PICK_UP: 'pickupShipment',
                ActType.DROP_OFF: 'deliverShipment',
                ActType.DELIVERY: 'delivery',
                ActType.RETURN: 'return',
                ActType.WAIT: 'wait',
                ActType.DRIVE: 'drive',
                ActType.IDLE: 'idle',
                }[act_type]

    def __str__(self):
        return self.get_string_from_type(self.type)

    def __repr__(self):
        return self.__str__()

class JspritAct(ActType):

    def __init__(self, type_=None, person_id=None, end_time=None, arrival_time=None):
        super(JspritAct, self).__init__(type_=type_)
        self.person_id = person_id
        self.end_time = end_time
        self.arrival_time = arrival_time

    def get_duration(self):
        return self.end_time - self.arrival_time

    def __str__(self):
        return 'Person{}, end_time {}, arrival_time {}' \
            .format(self.person_id, self.end_time, self.arrival_time)

    def __repr__(self):
        return self.__str__()

In [4]:
class JspritRoute(object):
    acts = None  # type: List[JspritAct]

    def __init__(self, vehicle_id=None, start_time=None, end_time=None, acts=None):
        self.vehicle_id = vehicle_id
        self.start_time = start_time
        self.end_time = end_time
        self.acts = acts
        
class JspritSolution(object):
    def __init__(self, cost, routes=None, unassigned=None):
        self.cost = cost
        self.routes = routes
        self.unassigned = unassigned
        self.modified_route = None

In [5]:
file = 'data/solution_day_sts_5300RINR.xml'
output = 'data/sjobo_initial_solution.xml'

In [6]:
solution = read_vrp_solution(file)

# take coordinates by shipment ID

In [7]:
sample = pd.read_excel('data/day_sts.xlsx')

In [8]:
kommun = geopandas.read_file('data/kommuns.geojson')
kommun = kommun.to_crs('EPSG:4326')

In [9]:
sample_points = [(Point(slon, slat), Point(elon, elat)) for slat, slon, elat, elon in zip(sample.start_latitude, sample.start_longitude ,sample.end_latitude, sample.end_longitude)] 

In [10]:
sj = []
for (start, end), i in zip(sample_points, range(10000)):
    kstart = kommun[kommun.contains(start)].kommun.iloc[0]
    kend = kommun[kommun.contains(end)].kommun.iloc[0]
    if kstart == '1265' and kend == '1265':
        sj.append(i)
    elif kstart == '1265' or kend == '1265':
        sj.append(i)

In [11]:
sjobo_people = sample.loc[sj]
sjobo_people = sjobo_people.set_index(['Avidentifierat Bokningsnummer'])

In [12]:
sjobo_people.head()

,område,fordonstyp,garage,datum,Avidentifierad kund,antal pass,Bok ensam,Villkor ensam,kund ensam,egen avg,...,Orter,Till Område,Till Område2,Områden,Timme,start_longitude,start_latitude,end_longitude,end_latitude,direct_time
Avidentifierat Bokningsnummer,,,,,,,,,,,,,,,,,,,,,
35544,4,Personbil,Sjöbo,20191111,679,4,NEJ,NEJ,JA,0,...,Bjärsjölagård-Sjöbo,4.0,4.0,4-4,8,13.682234,55.722382,13.704346,55.633126,1091.4
35545,4,Personbil,Ystad,20191111,2640,3,NEJ,NEJ,NEJ,30,...,Sjöbo-Sjöbo,4.0,4.0,4-4,11,13.714559,55.634324,13.715854,55.639613,217.8
35572,4,Klass 1 + trp,Ystad,20191111,10178,1,NEJ,NEJ,NEJ,64,...,Ystad-Sjöbo,4.0,4.0,4-4,10,13.823431,55.436185,13.696784,55.641145,1799.0
35583,5,Personbil,Hörby,20191111,10183,3,NEJ,NEJ,JA,97,...,Vollsjö-Kristianstad,4.0,3.0,4-3,7,13.790058,55.705442,14.182139,56.030608,3746.1
35616,3,Klass 1,Tollarp,20191111,6285,1,NEJ,NEJ,NEJ,110,...,Lövestad-Kristianstad,4.0,3.0,4-3,17,13.890064,55.652820,14.169410,56.030748,3673.2


# Select routes and shipments to put to initial solution

In [13]:
save_routes = []
save_shipments = []
for i, route in enumerate(solution.routes):
    ids = [act.person_id for act in route.acts]
    if any([i in sjobo_people.index for i in ids]):
        save_routes.append(i)
        save_shipments.extend(set(ids))
    

In [14]:
len(save_routes)

69

In [15]:
len(solution.routes)

440

In [16]:
len(save_shipments)

1253

# remove shipments from XML tree

In [17]:
ET.register_namespace('', 'http://www.w3schools.com')
namespace = {'xmlns': 'http://www.w3schools.com'}
tree = ET.parse(file)
root = tree.getroot()
shipments_element = root.find('xmlns:shipments', namespace)
shipment_elements = shipments_element.findall('xmlns:shipment', namespace)
for shipment_element in shipment_elements:
    if int(shipment_element.get('id')) not in save_shipments:
        shipments_element.remove(shipment_element)
        
solution_element = root.find('xmlns:solutions/xmlns:solution', namespace)
routes_element = solution_element.find('xmlns:routes', namespace)
route_elements = routes_element.findall('xmlns:route', namespace)
to_initial = []
for i, route in enumerate(route_elements):
#     if i not in save_routes:
#         routes_element.remove(route)
    if i in save_routes:
        to_initial.append(route)

solutions_element = root.find('xmlns:solutions', namespace)        
root.remove(solutions_element)
        
initial_routes_element = ET.SubElement(root, 'initialRoutes')
for route in to_initial:
    initial_routes_element.append(route)

# remove vehicles to restructure the problem to finite
vehicles_element = root.find('xmlns:vehicles', namespace)
vehicle_elements = vehicles_element.findall('xmlns:vehicle', namespace)
for v in vehicle_elements:
    vehicles_element.remove(v)
    
# write vehicles according to initial_solution ids
for i, route in enumerate(to_initial):
    vid = int(route.find('xmlns:vehicleId', namespace).text)
    nv = copy.deepcopy(vehicle_elements[vid])
    nvid = nv.find('xmlns:id', namespace)
    nvid.text = str(i)
    vehicles_element.append(nv)

#update vehicle id for initial routes
initial_route_elements = initial_routes_element.findall('xmlns:route', namespace)
for i,  route in enumerate(initial_route_elements):
    vid = route.find('xmlns:vehicleId', namespace)
    vid.text = str(i)
    
tree.write(output, xml_declaration=True, encoding="UTF-8")

In [18]:
save_routes

[1,
 2,
 5,
 7,
 36,
 37,
 38,
 39,
 46,
 47,
 48,
 49,
 52,
 53,
 54,
 64,
 65,
 66,
 72,
 88,
 104,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 119,
 120,
 121,
 124,
 127,
 138,
 139,
 142,
 143,
 145,
 184,
 191,
 193,
 194,
 195,
 197,
 216,
 217,
 218,
 231,
 239,
 261,
 276,
 285,
 296,
 297,
 306,
 310,
 322,
 349,
 350,
 351,
 360,
 362,
 385,
 386,
 395,
 400,
 409,
 422]

In [19]:
len(route_elements)

440

In [20]:
len(initial_routes_element.findall('xmlns:route', namespace))

69

In [21]:
len(route_elements)

440

In [22]:
ET.register_namespace('', 'http://www.w3schools.com')
namespace = {'xmlns': 'http://www.w3schools.com'}
tree = ET.parse(output)
root = tree.getroot()
shipments_element = root.find('xmlns:shipments', namespace)
shipment_elements = shipments_element.findall('xmlns:shipment', namespace)
len(shipment_elements)

1253

# Coordinates in play

In [23]:
sample = sample.set_index(['Avidentifierat Bokningsnummer'])

In [24]:
sample.head()

,område,fordonstyp,garage,datum,Avidentifierad kund,antal pass,Bok ensam,Villkor ensam,kund ensam,egen avg,...,Orter,Till Område,Till Område2,Områden,Timme,start_longitude,start_latitude,end_longitude,end_latitude,direct_time
Avidentifierat Bokningsnummer,,,,,,,,,,,,,,,,,,,,,
35517,2,Personbil avrop,Helsingborg,20191111,10164,2,NEJ,NEJ,NEJ,0,...,Helsingborg-Landskrona,2.0,2.0,2-2,9,12.703307,56.047238,12.846925,55.882490,1390.2
35530,2,Personbil avrop,Landskrona,20191111,1950,1,NEJ,NEJ,JA,0,...,Helsingborg-Helsingborg,2.0,2.0,2-2,12,12.729740,56.068052,12.703307,56.047238,357.9
35550,5,Klass 1 + trp,Eslöv,20191111,1209,2,NEJ,JA,NEJ,0,...,Malmö-Lund,8.0,0.0,8-0,9,13.003234,55.586852,13.195056,55.710888,1434.5
35563,7,Klass 3,Trelleborg,20191111,1209,2,JA,JA,NEJ,0,...,Malmö-Lund,8.0,0.0,8-0,12,13.003234,55.586852,13.195056,55.710888,1434.5
35553,6,Personbil,Lund,20191111,10172,2,NEJ,NEJ,JA,0,...,Bjärred-Lund,6.0,0.0,6-0,7,13.011490,55.727940,13.198575,55.713030,1063.9


In [25]:
sample.loc[save_shipments]

,område,fordonstyp,garage,datum,Avidentifierad kund,antal pass,Bok ensam,Villkor ensam,kund ensam,egen avg,...,Orter,Till Område,Till Område2,Områden,Timme,start_longitude,start_latitude,end_longitude,end_latitude,direct_time
Avidentifierat Bokningsnummer,,,,,,,,,,,,,,,,,,,,,
37379,1,Personbil,Ängelholm,20191111,10480,2,NEJ,NEJ,JA,136,...,Båstad-Lund,1.0,0.0,1-0,19,12.840827,56.430217,13.198575,55.713030,4809.4
38668,5,Klass 1,Hörby,20191111,10690,1,NEJ,NEJ,NEJ,64,...,Hörby-Eslöv,5.0,5.0,5-5,16,13.656175,55.848122,13.280647,55.840237,1832.9
36493,7,Klass 1,Svedala,20191111,700,1,NEJ,NEJ,NEJ,0,...,Ystad-Ystad,4.0,4.0,4-4,9,13.791871,55.436649,13.840908,55.442569,308.0
40227,6,Klass 1,Veberöd,20191111,10517,2,NEJ,NEJ,JA,77,...,Lund-Landskrona,0.0,2.0,0-2,17,13.198842,55.715639,12.827885,55.893823,1945.2
38579,4,Personbil,Ystad,20191111,10671,2,NEJ,NEJ,NEJ,30,...,Ystad-Ystad,4.0,4.0,4-4,9,13.837275,55.431820,13.900702,55.507616,711.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37214,4,Personbil,Ystad,20191111,1378,1,NEJ,NEJ,JA,30,...,Sjöbo-Sjöbo,4.0,4.0,4-4,10,13.708041,55.640491,13.704504,55.631807,257.6
39032,3,Personbil,Yngsjö,20191111,1075,1,NEJ,NEJ,NEJ,30,...,Sjöbo-Sjöbo,4.0,4.0,4-4,8,13.706222,55.641961,13.685545,55.634300,273.3
38989,2,Klass 4,Hyllinge,20191111,10492,2,NEJ,NEJ,NEJ,110,...,Lund-Helsingborg,0.0,2.0,0-2,9,13.196717,55.712171,12.707842,56.063717,2765.9


In [26]:
points = []
for shipment in sample.loc[save_shipments].itertuples():
    points.append((shipment.start_longitude, shipment.start_latitude))
    points.append((shipment.end_longitude, shipment.end_latitude))
points = set(points)

In [27]:
len(points)

1120

In [28]:
pd.DataFrame(points).to_csv('data/sj_routes.csv', header=False)